In [1]:
from ftplib import FTP
import os
#import shutil
import numpy as np
import zipfile
import pandas 
import geopandas
import time
from matplotlib import pyplot as plt
from shapely.ops import unary_union
from shapely.geometry import MultiPolygon
#import gzip
#from cycler import cycler

thisyear = 2018
#this_state = 'md'
equal_area_crs = {'init': 'epsg:2163'}  # An equal area projection: https://epsg.io/2163
#nDistrictsForBitmaskeration = 8


script_dir = '/home/idies/workspace/Storage/raddick/jordanraddick.com/'
thisdir = script_dir + 'gerrymandering/'
data_dir = '/home/idies/workspace/Temporary/raddick/census_scratch/acs5/{0:.0f}/estimates/'.format(thisyear)
shapefiledir = '/home/idies/workspace/Temporary/raddick/census_scratch/shapefiles/{0:.0f}/'.format(thisyear)
extras_dir = '/home/idies/workspace/Storage/raddick/census/extras/'

outdir = thisdir + '/districts_with_data/'

water_area_tol = 1 * 1000 * 1000
overlap_area_tract_tol = 22000
overlap_area_bg_tol = 4000
overlap_area_metro_tol = overlap_area_tract_tol
overlap_area_city_tol = overlap_area_metro_tol
#smallest tract in US is Cook County, Illinois Tract 307.02 (area = 22,094 m^2)
#smallest block group in US is Miami-Dade County, FL, Census Tract 2703, block group 7 (area = 4,436 m^2)

scale = 1
map_buffer = 0.25 # extra room on each edge of the maps, in degres

#plt.rc('axes', prop_cycle=default_cycler)

color_cycle = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
color_cycle = color_cycle + color_cycle
color_cycle = color_cycle + color_cycle

debug = 2
g = 0

print('ok')

ok


# Get congressional district shapefiles

## FTP down from ftp2.census.gov

In [2]:
# s = time.time()
# print('getting congressional district shapefiles from Census FTP...')
# os.chdir(shapefiledir+'CD/')

# ftp = FTP('ftp2.census.gov')
# ftp.login()
# #print(ftp.getwelcome())
# ftp.cwd('geo/tiger/TIGER{0:.0f}/CD/'.format(thisyear))
# #print(ftp.nlst())

# thefilename = 'tl_{0:.0f}_us_cd116.zip'.format(thisyear)
# #print(thefilename)
# with open(thefilename, 'wb') as f:
#     ftp.retrbinary('RETR {0:}'.format(thefilename), f.write)

# ftp.quit()
# #print('ok')

# print('unzipping...')

# thezipfile = zipfile.ZipFile(shapefiledir+'CD/tl_{0:.0f}_us_cd116.zip'.format(thisyear))
# thezipfile.extractall()
# thezipfile.close()
# os.remove(shapefiledir+'CD/tl_2018_us_cd116.zip')
# #os.listdir()
# e = time.time()
# g = g + (e-s)
# print('Got 1 file in {0:,.0f} seconds!'.format(e-s))
# #os.listdir()

## Load congressional district shapefiles into a GeoDataFrame

In [3]:
s = time.time()
print('reading congressional districts...')
cd_gdf = geopandas.read_file(shapefiledir+'CD/tl_2018_us_cd116.shp')
cd_gdf.loc[:, 'GEOID'] = cd_gdf['GEOID'].apply(lambda x: '50000US'+str(x))
#cd_gdf = cd_gdf.set_index('GEOID')

print('reading helpfel files...')
geo_summary_levels_df = pandas.read_csv(extras_dir+'geo_summary_levels.csv', index_col='SUMLEVEL')
statecodes_df = pandas.read_csv(extras_dir+'statecodes.csv', index_col='STATE')

print('converting to numeric and adding state names and setting index...')

for x in ['STATEFP', 'CD116FP']:
    cd_gdf.loc[:, x] = pandas.to_numeric(cd_gdf[x], errors='coerce', downcast='integer')
for x in ['CDSESSN', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON']:
    cd_gdf.loc[:, x] = pandas.to_numeric(cd_gdf[x], errors='coerce')

cd_gdf = cd_gdf.merge(statecodes_df.reset_index(), how='left', left_on='STATEFP', right_on='STATE').set_index('GEOID')


#cd_gdf = cd_gdf.set_index('GEOID')
e = time.time()
g = g + (e-s)
print('Read {0:,.0f} districts in {1:,.1f} seconds.'.format(len(cd_gdf), e-s))
#statecodes_df

#cd_gdf.apply(lambda row: '50000US{0:02d}{1:02d}'.format(int(row['STATEFP']), int(row['CD116FP'])), axis=1)
#cd_gdf[['CD116FP','NAMELSAD','LSAD','CDSESSN','STATE','STUSAB','STATE_NAME','STATENS']]


reading congressional districts...
reading helpfel files...
converting to numeric and adding state names and setting index...
Read 444 districts in 2.0 seconds.


## Get community-based statistical areas

In [4]:
s = time.time()

print('getting community-based statistical areas...')
cbsa_gdf = geopandas.read_file(shapefiledir+'CBSA/tl_2018_us_cbsa.shp')

print('removing CBSAs in Puerto Rico...')
cbsa_gdf = cbsa_gdf[cbsa_gdf['NAME'].apply(lambda x: ', PR' in x) == False]
cbsa_gdf = cbsa_gdf.set_index('GEOID')
cbsa_gdf = cbsa_gdf.sort_index()


print('reading OMB data for community-based statistical areas...')
cbsa_data_df = pandas.read_excel(extras_dir+'cbsa_list1_2020.xls', header=2)
cbsa_principal_cities_df = pandas.read_excel(extras_dir+'cbsa_list2_2020.xls', header=2)
print('chopping off non-data from bottom of dataframes...')
cbsa_data_df = cbsa_data_df.head(-4)
cbsa_principal_cities_df = cbsa_principal_cities_df.head(-4)
#cbsa_principal_cities_df = cbsa_principal_cities_df.rename(columns={'CBSA Code': 'GEOID'})
#cbsa_principal_cities_df = cbsa_principal_cities_df.set_index('GEOID')

e = time.time()
g = g + (e-s)
print('\nRead {0:,.0f} community-based statistical areas in {1:,.0f} seconds!'.format(len(cbsa_gdf), e-s))

#metro_areas_df.sample(1)




getting community-based statistical areas...
removing CBSAs in Puerto Rico...
reading OMB data for community-based statistical areas...
chopping off non-data from bottom of dataframes...

Read 933 community-based statistical areas in 2 seconds!


## Identify metropolitan statistical areas (MSAs)

In [5]:
metro_areas_gdf = cbsa_gdf.join(
    cbsa_principal_cities_df[
        (cbsa_principal_cities_df['Metropolitan/Micropolitan Statistical Area'] == 'Metropolitan Statistical Area')
        & (cbsa_principal_cities_df['CBSA Title'].apply(lambda x: x[-4:] != ', PR'))
    ][['CBSA Code', 'CBSA Title', 'Metropolitan/Micropolitan Statistical Area']].drop_duplicates().set_index('CBSA Code')
)

metro_areas_gdf = metro_areas_gdf[metro_areas_gdf['Metropolitan/Micropolitan Statistical Area'].notnull()]
metro_areas_gdf = metro_areas_gdf[[x for x in metro_areas_gdf.columns.tolist() if x != 'geometry'] + ['geometry']]

print('\nFound {0:,.0f} MSAs in {1:,.0f} seconds!'.format(len(metro_areas_gdf), e-s))


Found 381 MSAs in 2 seconds!


## Load places shapefiles

In [6]:
s = time.time()
place_gdf = geopandas.GeoDataFrame()
place_file_list = [shapefiledir+'PLACE/'+x for x in os.listdir(shapefiledir+'PLACE/') if x[-4:] == '.shp']
for i in range(0, len(place_file_list)):
    if (debug >= 2):
        if ((np.mod(i,10) == 0) | (i == len(place_file_list)-1)):
            print('\tReading file {0:,.0f} of {1:,.0f}...'.format(i+1, len(place_file_list)))
    place_gdf_i = geopandas.read_file(place_file_list[i])
    place_gdf = pandas.concat((place_gdf, place_gdf_i), axis=0, sort=False)


print('converting to numeric...')
place_gdf.loc[:, 'STATEFP'] = pandas.to_numeric(place_gdf['STATEFP'], errors='coerce')
place_gdf.loc[:, 'PLACEFP'] = pandas.to_numeric(place_gdf['PLACEFP'], errors='coerce')
print('setting GEOID as index...')
place_gdf = place_gdf.set_index("GEOID")
place_gdf = place_gdf.sort_index()

e = time.time()
g = g + (e-s)
print('Read {0:,.0f} places in {1:,.0f} seconds!'.format(len(place_gdf), e-s))


	Reading file 1 of 51...
	Reading file 11 of 51...
	Reading file 21 of 51...
	Reading file 31 of 51...
	Reading file 41 of 51...
	Reading file 51 of 51...
converting to numeric...
setting GEOID as index...
Read 29,321 places in 20 seconds!


## Identify principal cities for each metro area

### Metro areas with one principal city

In [7]:
s = time.time()
print('finding MSAs with only one principal city...')

value_counts_s = cbsa_principal_cities_df[
    cbsa_principal_cities_df['CBSA Code'].isin(metro_areas_gdf.index)
]['CBSA Code'].value_counts()

single_principal_city_list = value_counts_s[value_counts_s == 1].sort_index().index.tolist()

metro_areas_gdf.loc[single_principal_city_list]

metro_areas_gdf = metro_areas_gdf.assign(principal_city_placeid = np.nan)
metro_areas_gdf = metro_areas_gdf.assign(principal_cities_geometry = np.nan)

#metro_areas_gdf
metro_areas_gdf.loc[
    cbsa_principal_cities_df[
        cbsa_principal_cities_df['CBSA Code'].isin(single_principal_city_list)
    ]['CBSA Code'].tolist(), 'principal_city_placeid'] = cbsa_principal_cities_df[
        cbsa_principal_cities_df['CBSA Code'].isin(single_principal_city_list)
        ][
            ['FIPS State Code', 'FIPS Place Code']
        ].apply(lambda row: '{0:02d}{1:05d}'.format(int(row['FIPS State Code']), int(row['FIPS Place Code'])), axis=1).values


metro_areas_gdf.loc[
    metro_areas_gdf[metro_areas_gdf['principal_city_placeid'].notnull()].index
, 'principal_cities_geometry'] = place_gdf.loc[
    metro_areas_gdf[metro_areas_gdf['principal_city_placeid'].notnull()]['principal_city_placeid']
].geometry.values

e = time.time()
g = g + (e-s)

print('added principal city geometries for {0:,.0f} MSAs with a single principal city in {1:,.1f} seconds!'.format(len(metro_areas_gdf[metro_areas_gdf['principal_city_placeid'].notnull()]), e-s))


finding MSAs with only one principal city...
added principal city geometries for 244 MSAs with a single principal city in 0.1 seconds!


### Metro areas with multiple principal cities

In [8]:
s = time.time()
print('finding MSAs with multiple principal cities...')

need_these_metro_areas_list = metro_areas_gdf[metro_areas_gdf['principal_city_placeid'].isnull()].index.tolist()
newgdf = geopandas.GeoDataFrame(data=None, columns=['geometry'], crs=metro_areas_gdf.crs, geometry='geometry')

cnt = 0
for this_metro_area in need_these_metro_areas_list:
    cnt = cnt + 1
    if ((np.mod(cnt,20) == 0) | (cnt == len(need_these_metro_areas_list))):
        print('Checking metro area {0:,.0f} of {1:,.0f}...'.format(cnt, len(need_these_metro_areas_list)))
    the_principal_city_place_id_list = cbsa_principal_cities_df[
        cbsa_principal_cities_df['CBSA Code'] == this_metro_area].apply(lambda row: 
                                                          '{0:02d}{1:05d}'.format(int(row['FIPS State Code']), int(row['FIPS Place Code'])), axis=1).tolist()
    this_metro_area_principal_city_geolist = []
    for this_principal_city_id in the_principal_city_place_id_list:
        if (place_gdf.loc[this_principal_city_id].geometry.type == 'Polygon'):
            this_metro_area_principal_city_geolist.append(place_gdf.loc[this_principal_city_id].geometry)
        else:
            for x in place_gdf.loc[this_principal_city_id].geometry:
                if (x.type == 'Polygon'):
                    this_metro_area_principal_city_geolist.append(x)
                else:
                    print(x.type)
    combined_geo = unary_union(this_metro_area_principal_city_geolist)
    newgdf.loc[this_metro_area, 'geometry'] = combined_geo
#newgdf.index.name = 'GEOID'
metro_areas_gdf.loc[newgdf.index, 'principal_cities_geometry'] = newgdf.geometry
metro_areas_gdf = metro_areas_gdf.drop('principal_city_placeid', axis=1)

# print('backing up...')
# cd_gdf_bk = cd_gdf
# metro_areas_gdf_bk = metro_areas_gdf

e = time.time()
g = g + (e-s)

print('Done in {0:,.0f} seconds!'.format(e-s))
#print('ok')
                    

finding MSAs with multiple principal cities...
Checking metro area 20 of 137...
Checking metro area 40 of 137...
Checking metro area 60 of 137...
Checking metro area 80 of 137...
Checking metro area 100 of 137...
Checking metro area 120 of 137...
Checking metro area 137 of 137...
Done in 22 seconds!


In [9]:
s = time.time()

print('keeping only the 435 voting members...')
cd_gdf = cd_gdf[
    (cd_gdf['NAMELSAD'].apply(lambda x: ('Congressional District' in x) == True))
    & (cd_gdf['CD116FP'].notnull())
]

cd_gdf = cd_gdf.assign(pct_metro_area_overlap = np.nan)
cd_gdf = cd_gdf.assign(pct_city_area_overlap = np.nan)
cd_gdf = cd_gdf.assign(type = np.nan)

print('\n')
print('calculating overlap between congressional districts and metro areas...')
print('Considering {0:,.0f} congressional districts...'.format(len(cd_gdf)))

i = 0
for ix, thisrow in cd_gdf.sort_values(by=['STATE_NAME', 'CD116FP']).iterrows():
    i = i + 1
    if ((np.mod(i,50) == 0) | (i == 435)):
        print('Reading district {0:,.0f} of 435...'.format(i))
#     print('{0:}-{1:.0f}...'.format(
#         thisrow['STATE_NAME'], 
#         thisrow['CD116FP']
#     ))
    total_metro_area_overlap = 0
    for jx, thatrow in metro_areas_gdf[metro_areas_gdf.geometry.apply(lambda x: x.intersects(thisrow.geometry))].iterrows():
        intersector = thisrow.geometry.intersection(thatrow.geometry)
        intersector_area_sq_m = geopandas.GeoSeries(intersector, crs=cd_gdf.crs).to_crs(equal_area_crs).geometry.values[0].area
        if (intersector_area_sq_m >= overlap_area_metro_tol):
            total_metro_area_overlap = total_metro_area_overlap + intersector_area_sq_m
    cd_gdf.loc[ix, 'pct_metro_area_overlap'] = total_metro_area_overlap / (thisrow['ALAND'] + thisrow['AWATER'])
#    metro_areas_gdf.loc[ix, 'pct_metro_area'] = intersector_area_sq_m / (thisrow['ALAND'] + thisrow['AWATER'])
#metro_areas_gdf

print('\n')
print('calculating overlap between congressional districts and principal cities...')
i = 0
metro_areas_gdf = metro_areas_gdf.set_geometry('principal_cities_geometry')
for ix, thisrow in cd_gdf.iterrows():
    i = i + 1
    if ((np.mod(i,50) == 0) | (i == 435)):
        print('Reading district {0:,.0f} of 435...'.format(i))
#     print('{0:}-{1:.0f}...'.format(
#         thisrow['STATE_NAME'], 
#         thisrow['CD116FP']
#     ))
    total_city_overlap = 0
    for jx, thatrow in metro_areas_gdf[metro_areas_gdf.geometry.apply(lambda x: x.intersects(thisrow.geometry))].iterrows():
        city_intersector = thisrow.geometry.intersection(thatrow.geometry)
        city_intersector_area_sq_m = geopandas.GeoSeries(city_intersector, crs=cd_gdf.crs).to_crs(equal_area_crs).geometry.values[0].area
        if (city_intersector_area_sq_m >= overlap_area_city_tol):
            total_city_overlap = total_city_overlap + city_intersector_area_sq_m
    cd_gdf.loc[ix, 'pct_city_area_overlap'] = total_city_overlap / (thisrow['ALAND'] + thisrow['AWATER'])

#cd_gdf = cd_gdf['pct_metro_area_overlap'].fillna(0)
#cd_gdf = cd_gdf['pct_city_overlap'].fillna(0)

metro_areas_gdf = metro_areas_gdf.set_geometry('geometry')

print('\n')
print('identifying urban/suburban/rural based on pct_metro_area_overlap, pct_city_area_overlap...')

cd_gdf = cd_gdf.assign(type = np.nan)

cd_gdf.loc[cd_gdf['pct_city_area_overlap'] > .95, 'type'] = 'Urban'
cd_gdf.loc[cd_gdf['pct_metro_area_overlap'] <= .5, 'type'] = 'Rural'

cd_gdf.loc[
    (cd_gdf['pct_city_area_overlap'] <= .95)
    & (cd_gdf['pct_metro_area_overlap'] > .5)
    , 'type'] = 'Suburban'

e = time.time()
g = g + (e-s)

print('Got district urban/suburban/rural for {0:,.0f} districts in {1:,.0f} minutes {2:,.0f} seconds!'.format(len(cd_gdf), np.floor((e-s)/60), np.floor((e-s)%60)))



print(cd_gdf.groupby('type').size())



keeping only the 435 voting members...


calculating overlap between congressional districts and metro areas...
Considering 435 congressional districts...
Reading district 50 of 435...
Reading district 100 of 435...
Reading district 150 of 435...
Reading district 200 of 435...
Reading district 250 of 435...
Reading district 300 of 435...
Reading district 350 of 435...
Reading district 400 of 435...
Reading district 435 of 435...


calculating overlap between congressional districts and principal cities...
Reading district 50 of 435...
Reading district 100 of 435...
Reading district 150 of 435...
Reading district 200 of 435...
Reading district 250 of 435...
Reading district 300 of 435...
Reading district 350 of 435...
Reading district 400 of 435...
Reading district 435 of 435...


identifying urban/suburban/rural based on pct_metro_area_overlap, pct_city_area_overlap...
Got district urban/suburban/rural for 435 districts in 6 minutes 9 seconds!
type
Rural       137
Suburban    115
Urban

In [10]:
cd_gdf[cd_gdf['type'] == 'Rural'][['STATE_NAME', 'CD116FP']
                                 ].sort_values(['STATE_NAME', 'CD116FP'])[125:]

,STATE_NAME,CD116FP
GEOID,,
50000US5302,Washington,2.0
50000US5303,Washington,3.0
50000US5304,Washington,4.0
50000US5306,Washington,6.0
50000US5401,West Virginia,1.0
50000US5402,West Virginia,2.0
50000US5403,West Virginia,3.0
50000US5503,Wisconsin,3.0
50000US5505,Wisconsin,5.0


In [12]:
print('Saving districts...')
cd_gdf.reset_index().to_file(outdir+'cd116_with_areas_and_types_435.shp')
print('DONE! Total time: {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor(g/60), np.floor(g%60)))



Saving districts...
DONE! Total time: 6 minutes 55 seconds!


In [13]:
z = geopandas.read_file(outdir+'cd116_with_areas_and_types_435.shp')
z

,GEOID,STATEFP,CD116FP,NAMELSAD,LSAD,CDSESSN,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,STATE,STUSAB,STATE_NAME,STATENS,pct_metro_,pct_city_a,type,geometry
0,50000US2707,27,7.0,Congressional District 7,C2,116,G5200,N,86580712408,5058381039,46.553020,-95.675905,27,MN,Minnesota,662849,0.125114,0.085973,Rural,"POLYGON ((-97.23909 48.96826, -97.23909 48.968..."
1,50000US2702,27,2.0,Congressional District 2,C2,116,G5200,N,6314464923,243358361,44.478968,-92.853042,27,MN,Minnesota,662849,0.622125,0.405532,Suburban,"POLYGON ((-93.91131 44.54651, -93.91024 44.548..."
2,50000US2706,27,6.0,Congressional District 6,C2,116,G5200,N,7460634590,408723230,45.336807,-93.852055,27,MN,Minnesota,662849,0.997696,0.333439,Suburban,"POLYGON ((-94.64041 45.37072, -94.64039 45.371..."
3,50000US2708,27,8.0,Congressional District 8,C2,116,G5200,N,72281499178,12448503736,47.250337,-92.963870,27,MN,Minnesota,662849,0.283819,0.235771,Rural,"POLYGON ((-95.18322 47.41277, -95.16598 47.412..."
4,50000US1701,17,1.0,Congressional District 1,C2,116,G5200,N,669256000,2067510,41.546671,-87.835747,17,IL,Illinois,1779784,0.998580,0.998580,Urban,"POLYGON ((-88.13677 41.42285, -88.13633 41.422..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,50000US3613,36,13.0,Congressional District 13,C2,116,G5200,N,26316818,7203018,40.838862,-73.932982,36,NY,New York,1779796,0.998744,0.998744,Urban,"POLYGON ((-73.97062 40.81700, -73.97011 40.817..."
431,50000US3626,36,26.0,Congressional District 26,C2,116,G5200,N,567624846,35592405,42.961826,-78.840983,36,NY,New York,1779796,0.998250,0.998250,Urban,"POLYGON ((-79.07537 43.08135, -79.07400 43.083..."
432,50000US3622,36,22.0,Congressional District 22,C2,116,G5200,N,13150495474,386868974,42.870694,-75.662664,36,NY,New York,1779796,0.730834,0.468466,Suburban,"POLYGON ((-76.47265 42.00007, -76.47213 42.000..."
433,50000US3609,36,9.0,Congressional District 9,C2,116,G5200,N,40269800,499820,40.642184,-73.947201,36,NY,New York,1779796,0.998790,0.998790,Urban,"POLYGON ((-73.98784 40.65982, -73.98750 40.660..."
